## 88. Spark Catalyst Optimizer:

In [1]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session

spark = spark_session()

25/11/06 21:41:47 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/11/06 21:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/06 21:41:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/06 21:41:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df = spark.read.parquet("./data/vuelos.parquet")
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [3]:
df.show(truncate=False)

25/11/06 21:45:20 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [4]:
from pyspark.sql.functions import col

In [5]:
nuevo_df = (
    df.filter(col("MOnTH").isin(6,7,8))
    .withColumn("dis_tiempo_aire", col("DISTANCE")/col("AIR_TIME"))
).select(
    col("AIRLINE"),
    col("dis_tiempo_aire"),
).where(col("AIRLINE").isin("AA", "DL", "AS"))

In [7]:
nuevo_df.explain(True)

== Parsed Logical Plan ==
'Filter 'AIRLINE IN (AA,DL,AS)
+- Project [AIRLINE#4, dis_tiempo_aire#188]
   +- Project [YEAR#0, MONTH#1, DAY#2, DAY_OF_WEEK#3, AIRLINE#4, FLIGHT_NUMBER#5, TAIL_NUMBER#6, ORIGIN_AIRPORT#7, DESTINATION_AIRPORT#8, SCHEDULED_DEPARTURE#9, DEPARTURE_TIME#10, DEPARTURE_DELAY#11, TAXI_OUT#12, WHEELS_OFF#13, SCHEDULED_TIME#14, ELAPSED_TIME#15, AIR_TIME#16, DISTANCE#17, WHEELS_ON#18, TAXI_IN#19, SCHEDULED_ARRIVAL#20, ARRIVAL_TIME#21, ARRIVAL_DELAY#22, DIVERTED#23, ... 8 more fields]
      +- Filter MOnTH#1 IN (6,7,8)
         +- Relation [YEAR#0,MONTH#1,DAY#2,DAY_OF_WEEK#3,AIRLINE#4,FLIGHT_NUMBER#5,TAIL_NUMBER#6,ORIGIN_AIRPORT#7,DESTINATION_AIRPORT#8,SCHEDULED_DEPARTURE#9,DEPARTURE_TIME#10,DEPARTURE_DELAY#11,TAXI_OUT#12,WHEELS_OFF#13,SCHEDULED_TIME#14,ELAPSED_TIME#15,AIR_TIME#16,DISTANCE#17,WHEELS_ON#18,TAXI_IN#19,SCHEDULED_ARRIVAL#20,ARRIVAL_TIME#21,ARRIVAL_DELAY#22,DIVERTED#23,... 7 more fields] parquet

== Analyzed Logical Plan ==
AIRLINE: string, dis_tiempo_aire: 